# Stable Diffusion: Accelerated

<figure>
<img src="assets/SD_Diagram.png", width="600">
<figcaption align = "center"> Figure 1. Architecture Diagram of Stable Diffusion Components license: MIT Diffusion architecture.png dataset </figcaption>
</figure>


# Overview

I use Stable Diffusion to synthesize aerial photos for a number of reasons:

1) Real images that are free to download are time consuming to obtain.
2) Real images that are easy to obtain via nice API's, are typically NOT free.
3) Time constraints limit the number I am willing to download.
4) Distribution of real data to attendees is problematic from a usage license stand point.
5) Synthetic images can be generated in bulk easily
6) There is a key learning I want to share with attendees regarding optimizing Stable Diffusion using the Intel(R) Extensions for PyTorch*

   
Stable Diffusion is a technique that uses a special kind of diffusion model developed by the CompVis group at LMU Munich. This model helps improve images by reducing noise. It has three main parts: a variational autoencoder (VAE) that compresses the image, a U-Net that removes noise, and an optional text encoder. The VAE makes the image smaller and more meaningful, while the U-Net removes noise. The process involves applying Gaussian noise and then removing it. The VAE decoder completes the process by turning the compressed image back into a clear picture.



# What can it do?

<figure>
<img src="assets/X-Y_plot_of_algorithmically-generated_AI_art_of_European-style_castle_in_Japan_demonstrating_DDIM_diffusion_steps.png", width="600">
<figcaption align = "center"> Figure 2. Image courtesy of Benlisquare at wikipedia: Stable Diffusion (File:X-Y plot of algorithmically-generated AI art of European-style castle in Japan demonstrating DDIM diffusion steps.png) </figcaption>
</figure>

# Stable Diffusion for Synthetic Aerial Photos

In this project we downloaded 10 images from US government sources for NAIP/DOQQ images with two classes in mind - photos which will be marred in the future by forest fires, and photos which will remain un-phased by forest fires in the next two years (referencing 2016/2017).

The code below was taken from Hugging face and optimized by our Intel team!

It uses Intel(R) Extension for PyTorch* (IPEX) to allow us to target an Intel® Data Center GPU Max Series GPU on the Intel(r) Developer Cloud Beta.

The interesting take away here is that the Stable Diffusion model is not a single model but rather a pipeline of 4 standalone models and these optimization techniques get applied to all models in the pipeline!

Significant speedups come from applying IPEX. Try it yourself!

### Exercise 3: Modify the code below to use IPEX to optimize each stage of the SD pipeline

The code from retrieved from Hugging Face has not had IPEX optimizations applied to each stage of the pipeline and will take ~ 17 minutes to run. To dramatically speed this up, complete Exercise 3!

Find Exercise 3 below and un-comment the code or else replace with this code:
```python
        for attr in dir(pipeline):
            if isinstance(getattr(pipeline, attr), nn.Module):
                setattr(
                    pipeline,
                    attr,
                    ipex.optimize(
                        getattr(pipeline, attr).eval(),
                        dtype=pipeline.text_encoder.dtype,
                        inplace=True,
                    ),
                 )
```

### Exercise 4: Modify the text to generate custom images

Add a few text descriptor sentences to help generate better images

- Hint: try:

```python
fire_variations = [
    "vegetation dark green and brown low saturation image mountainous slopes early morning clear skies",
    "vegetation dark green and brown low saturation image mountainous slopes late afternoon clear skies",
    "vegetation dark green and brown low saturation image mountainous slopes mid-day no clouds",
    "vegetation dark green and brown low saturation image mountainous slopes mid-day minimal light sirius cloud cover",
    "vegetation dark green and brown low saturation image mountainous slopes smoky conditions",
    "vegetation dark green and brown low saturation image mountainous slopes late afternoon minimal light sirius cloud cover",
]
no_fire_variations = [
    "early morning with clear skies vegetation bright green vibrant saturation",
    "no signs of fire vegetation bright green vibrant saturation",
    "day vegetation bright green vibrant saturation",
    "late afternoon with clear skies vegetation bright green vibrant saturation",
    "mid-day with clear skies vegetation bright green vibrant saturation",
    "with dense vegetation vegetation bright green vibrant saturation",
    "with sparse vegetation vegetation bright green vibrant saturation",
]
```

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")

import random
import requests
import torch
import intel_extension_for_pytorch as ipex
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline
import torch.nn as nn
import time
from typing import List, Dict, Tuple

class Img2ImgModel:
    """
    This class creates a model for transforming images based on given prompts.
    """

    def __init__(
        self,
        model_id_or_path: str,
        device: str = "xpu",
        torch_dtype: torch.dtype = torch.float16,
        optimize: bool = True,
    ) -> None:
        """
        Initialize the model with the specified parameters.

        Args:
            model_id_or_path (str): The ID or path of the pre-trained model.
            device (str, optional): The device to run the model on. Defaults to "xpu".
            torch_dtype (torch.dtype, optional): The data type to use for the model. Defaults to torch.float16.
            optimize (bool, optional): Whether to optimize the model. Defaults to True.
        """
        self.device = device
        self.pipeline = self._load_pipeline(model_id_or_path, torch_dtype)
        if optimize:
            start_time = time.time()
            print("Optimizing the model...")
            self.optimize_pipeline()
            print(
                "Optimization completed in {:.2f} seconds.".format(
                    time.time() - start_time
                )
            )

    def _load_pipeline(
        self, model_id_or_path: str, torch_dtype: torch.dtype
    ) -> StableDiffusionImg2ImgPipeline:
        """
        Load the pipeline for the model.

        Args:
            model_id_or_path (str): The ID or path of the pre-trained model.
            torch_dtype (torch.dtype): The data type to use for the model.

        Returns:
            StableDiffusionImg2ImgPipeline: The loaded pipeline.
        """
        print("Loading the model...")
        pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_id_or_path, torch_dtype=torch_dtype
        )
        pipeline = pipeline.to(self.device)
        print("Model loaded.")
        return pipeline

    def _optimize_pipeline(
        self, pipeline: StableDiffusionImg2ImgPipeline
    ) -> StableDiffusionImg2ImgPipeline:
        """
        Optimize the pipeline of the model.

        Args:
            pipeline (StableDiffusionImg2ImgPipeline): The pipeline to optimize.

        Returns:
            StableDiffusionImg2ImgPipeline: The optimized pipeline.
        """

############################### Exercise 3 #####################################
        ########### uncomment the lines below #########################
        
        # for attr in dir(pipeline):
        #     if isinstance(getattr(pipeline, attr), nn.Module):
        #         setattr(
        #             pipeline,
        #             attr,
        #             ipex.optimize(
        #                 getattr(pipeline, attr).eval(),
        #                 dtype=pipeline.text_encoder.dtype,
        #                 inplace=True,
        #             ),
        #          )

##################################################################################
        return pipeline

    def optimize_pipeline(self) -> None:
        """
        Optimize the pipeline of the model.
        """
        self.pipeline = self._optimize_pipeline(self.pipeline)

    def get_image_from_url(self, url: str, path: str) -> Image.Image:
        """
        Get an image from a URL or from a local path if it exists.

        Args:
            url (str): The URL of the image.
            path (str): The local path of the image.

        Returns:
            Image.Image: The loaded image.
        """
        if os.path.exists(path):
            img = Image.open(path).convert("RGB")
        else:
            response = requests.get(url)
            if response.status_code != 200:
                raise Exception(
                    f"Failed to download image. Status code: {response.status_code}"
                )
            if not response.headers["content-type"].startswith("image"):
                raise Exception(
                    f"URL does not point to an image. Content type: {response.headers['content-type']}"
                )
            img = Image.open(BytesIO(response.content)).convert("RGB")
            img.save(path)
        img = img.resize((768, 512))
        return img

    @staticmethod
    def random_sublist(lst):
        sublist = []
        for _ in range(random.randint(1, len(lst))):
            item = random.choice(lst)
            sublist.append(item)
        return sublist

    def generate_images(
        self,
        prompt: str,
        image_url: str,
        class_name: str,
        seed_image_identifier: str,
        variations: List[str],
        num_images: int = 5,
        strength: float = 0.75,
        guidance_scale: float = 7.5,
        save_path: str = "output",
        seed_path: str = "intput",
    ) -> List[Image.Image]:
        """
        Generate images based on the provided prompt and variations.

        Args:
            prompt (str): The base prompt for the generation.
            image_url (str): The URL of the seed image.
            class_name (str): The class of the image (e.g. "fire" or "no_fire").
            seed_image_identifier (str): The identifier of the seed image.
            variations (List[str]): The list of variations to apply to the prompt.
            num_images (int, optional): The number of images to generate. Defaults to 5.
            strength (float, optional): The strength of the transformation. Defaults to 0.75.
            guidance_scale (float, optional): The scale of the guidance. Defaults to 7.5.
            save_path (str, optional): The path to save the generated images. Defaults to "output".
            seed_path (str, optional): The path to save the input images. Defaults to "input".

        Returns:
            List[Image.Image]: The list of generated images.
        """
        input_image_path = f"{seed_path}/{seed_image_identifier}.png"
        init_image = self.get_image_from_url(image_url, input_image_path)
        images = []
        for i in range(num_images):
            variation = variations[i % len(variations)]
            final_prompt = f"{prompt} {variation}"
            image = self.pipeline(
                prompt=final_prompt,
                image=init_image,
                strength=strength,
                guidance_scale=guidance_scale,
            ).images
            output_image_path = os.path.join(
                save_path,
                f"{seed_image_identifier}_{'_'.join(variation.split())}_{i}.png",
            )
            image[0].save(output_image_path)
            images.append(image)
        return images

model_id = "runwayml/stable-diffusion-v1-5"
base_prompt = "A close image to this original satellite image with slight change in location"


############### Exercise 3: add text descriptions such as those in the hint #################
fire_variations = [
    "day vegetation dull with little saturation"
]
no_fire_variations = [
    "day vegetation bright green vibrant saturation",
]
###############################################################################################


image_urls = {
    "fire": [
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/Fire/m_3912105_sw_10_h_20160713.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/Fire/m_3912113_sw_10_h_20160713.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/Fire/m_3912114_se_10_h_20160806.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/Fire/m_3912120_ne_10_h_20160713.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/Fire/m_4012355_se_10_h_20160713.png?raw=true",
    ],
    "no_fire": [
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/NoFire/m_3912045_ne_10_h_20160712.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/NoFire/m_3912057_sw_10_h_20160711.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/NoFire/m_3912142_sw_10_h_20160711.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/NoFire/m_3912343_se_10_h_20160529.png?raw=true",
        "https://github.com/rahulunair/ForestFirePrediction/blob/main/data/output/train/NoFire/m_4012241_se_10_h_20160712.png?raw=true",
    ],
}


# Intel® Data Center GPU Max Series Optimized with Intel Extension for PyTorch*

In [ ]:
timing = {}

In [ ]:
Optimize = True

model = Img2ImgModel(model_id, device="xpu", optimize=Optimize)
num_images = 2
gen_img_count = 0

try:
    trips = 0
    start_time = time.time()
    for class_name, urls in image_urls.items():
        for url in urls:
            seed_image_identifier = os.path.basename(url).split(".")[0]
            input_dir = f"./input/{class_name}"
            output_dir = f"./output/{class_name}"
            os.makedirs(input_dir, exist_ok=True)
            os.makedirs(output_dir, exist_ok=True)
            variations = (
                fire_variations if class_name == "fire" else no_fire_variations
            )
            model.generate_images(
                base_prompt,
                url,
                class_name,
                seed_image_identifier,
                variations=variations,
                save_path=output_dir,
                seed_path=input_dir,
                num_images=num_images,
            )
            gen_img_count += num_images
            trips += 1
            print("input image",trips)
except KeyboardInterrupt:
    print("\nUser interrupted image generation...")
finally:
    timing["xpu IPEX Optimized"] = time.time() - start_time
    print(
        f"Complete generating {gen_img_count} images in {'/'.join(output_dir.split('/')[:-1])} in {time.time() - start_time:.2f} seconds."
    )

# Intel® Data Center GPU Max Series GPU Without IPEX

* Run this when you go to lunch, it takes ~ 17 minutes.  You should get similar results as below, your mileage may vary, depending on how busy Intel Developer Cloud (IDC) is at the time of your workshop.

![ipex_vs_non_ipex_torch_inference](assets/ipex_sd_results.png)



In [ ]:
Optimize = False

model = Img2ImgModel(model_id, device="xpu", optimize=Optimize)
num_images = 2
gen_img_count = 0

try:
    trips = 0
    start_time = time.time()
    for class_name, urls in image_urls.items():
        for url in urls:
            seed_image_identifier = os.path.basename(url).split(".")[0]
            input_dir = f"./input/{class_name}"
            output_dir = f"./output/{class_name}"
            os.makedirs(input_dir, exist_ok=True)
            os.makedirs(output_dir, exist_ok=True)
            variations = (
                fire_variations if class_name == "fire" else no_fire_variations
            )
            model.generate_images(
                base_prompt,
                url,
                class_name,
                seed_image_identifier,
                variations=variations,
                save_path=output_dir,
                seed_path=input_dir,
                num_images=num_images,
            )
            gen_img_count += num_images
            trips += 1
            print("input image",trips)
except KeyboardInterrupt:
    print("\nUser interrupted image generation...")
finally:
    timing["xpu NOT Optimized"] = time.time() - start_time
    print(
        f"Complete generating {gen_img_count} images in {'/'.join(output_dir.split('/')[:-1])} in {time.time() - start_time:.2f} seconds."
    )

# Plot the timing results

In [ ]:
timing = dict(sorted(timing.items(), key=lambda item: item[1]))
nam = [k for k,v in [t for t in timing.items()]]
tim = [v for k,v in [t for t in timing.items()]]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.title("Time taken to generate images in seconds",fontsize=12)
plt.ylabel("Time in seconds",fontsize=12)
plt.xlabel("Various optimizations",fontsize=14)
plt.grid(True)
plt.xticks(rotation=-60)
plt.bar(x = list(timing.keys()), height= list(timing.values()), align='center',tick_label=list(timing.keys()))
print(f"Acceleration from IPEX {max(tim)/min(tim):.1f}X")
plt.savefig('ipex_sd_results.png')

## Notices and Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure. 

Your costs and results may vary. 

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. Other names and brands may be claimed as the property of others. 